In [6]:
import re
from collections import namedtuple

In [28]:
from typing import NamedTuple

In [83]:
import operator as op
from functools import partial

class Amounts(NamedTuple):
    ore: int = 0
    clay: int = 0
    obsidian: int = 0
    geode: int = 0
    
    def __add__(self, other: Amounts): return Amounts(*map(op.add,self,other))
    def __sub__(self, other: Amounts): return Amounts(*map(op.sub,self,other))
    def __mul__(self, other):
        if isinstance(other, Amounts):
            return Amounts(*map(op.mul,self,other))
        else:
            return Amounts(*map(partial(op.mul,other),self))
            


In [84]:
BP = namedtuple("Blueprint", "ore,clay,obsidian,geode")

In [85]:
def parse(fname):
    for line in open(fname):
        if line.strip():
            _,*ns= re.findall('[0-9]+', line)
            ns = map(int,ns)
            yield BP(ore=Amounts(ore=next(ns)), 
                               clay=Amounts(ore=next(ns)),
                               obsidian=Amounts(ore=next(ns),clay=next(ns)),
                                geode=Amounts(ore=next(ns),obsidian=next(ns)))
            
bps = list(parse("test"))
bps

[Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=2, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=3, clay=14, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=7, geode=0)),
 Blueprint(ore=Amounts(ore=2, clay=0, obsidian=0, geode=0), clay=Amounts(ore=3, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=3, clay=8, obsidian=0, geode=0), geode=Amounts(ore=3, clay=0, obsidian=12, geode=0))]

In [125]:
def evaluate(moves: list, bp: BP, t=24):
    minerals = Amounts()
    robots = Amounts(ore=1)
    breakpoint()
    while t > 0:
        #build robots
        move = moves.pop(0) if len(moves) > 0 else None
        if move:
            for i in range(4):
                minerals = minerals - bp[i]*move[i]
            
            if any(m < 0 for m in minerals):
                raise ValueError(minerals)
            
            new_robots = robots + move
            
        #collect
        minerals = minerals + robots
        
        #complete robots
        if move:
            robots = new_robots
        
        print(24-t+1,minerals)
        t-=1
    return minerals

moves=[
    None,None,Amounts(clay=1),
    None,Amounts(clay=1),None,
    Amounts(clay=1),None,None,
    None,Amounts(obsidian=1),Amounts(clay=1),
    None,None,Amounts(obsidian=1),
    None,None,Amounts(geode=1),
    None,None,Amounts(geode=1),
    None,None,None
]

evaluate(moves,bps[0])

1 Amounts(ore=1, clay=0, obsidian=0, geode=0)
2 Amounts(ore=2, clay=0, obsidian=0, geode=0)
3 Amounts(ore=1, clay=0, obsidian=0, geode=0)
4 Amounts(ore=2, clay=1, obsidian=0, geode=0)
5 Amounts(ore=1, clay=2, obsidian=0, geode=0)
6 Amounts(ore=2, clay=4, obsidian=0, geode=0)
7 Amounts(ore=1, clay=6, obsidian=0, geode=0)
8 Amounts(ore=2, clay=9, obsidian=0, geode=0)
9 Amounts(ore=3, clay=12, obsidian=0, geode=0)
10 Amounts(ore=4, clay=15, obsidian=0, geode=0)
11 Amounts(ore=2, clay=4, obsidian=0, geode=0)
12 Amounts(ore=1, clay=7, obsidian=1, geode=0)
13 Amounts(ore=2, clay=11, obsidian=2, geode=0)
14 Amounts(ore=3, clay=15, obsidian=3, geode=0)
15 Amounts(ore=1, clay=5, obsidian=4, geode=0)
16 Amounts(ore=2, clay=9, obsidian=6, geode=0)
17 Amounts(ore=3, clay=13, obsidian=8, geode=0)
18 Amounts(ore=2, clay=17, obsidian=3, geode=0)
19 Amounts(ore=3, clay=21, obsidian=5, geode=1)
20 Amounts(ore=4, clay=25, obsidian=7, geode=2)
21 Amounts(ore=3, clay=29, obsidian=2, geode=3)
22 Amounts(or

Amounts(ore=6, clay=41, obsidian=8, geode=9)